In [1]:
import pickle
import numpy as np
import pandas as pd
import math
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
from bisect import bisect_left
from scipy.stats import norm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
# import torch.optim as optim
from scipy import sparse
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import pickle
import os

In [ ]:
TIME_WINDOW = 300
longest_window = 600

regression  = True
number_selection = 1 
BATCH_SIZE = 64
device = torch.device('cuda:{}'.format(3))

def data_augumentation(train_data, train_label,train_mask,train_down_mat,aug_times = 10):
    new_train_data = []
    new_train_label = []
    
    for i in range(len(train_label)):
        if(train_label[i]>=5):
            new_train_data.extend([train_data[i] for j in range(aug_times)])
            new_train_label.extend([train_label[i] for j in range(aug_times)])
    
    new_train_data = np.array(new_train_data)
    new_train_label = np.array(new_train_label)
    
    train_data = np.concatenate((train_data,new_train_data), axis = 0)
    train_label = np.concatenate((train_label, new_train_label), axis = 0)

    return train_data, train_label
def take_closest(myList, myNumber):
    """
    Assumes myList is sorted. Returns closest value to myNumber.

    If two numbers are equally close, return the smallest number.
    """
    pos = bisect_left(myList, myNumber)
    if pos == 0:
        return 0
    if pos == len(myList):
        return len(myList) - 1
    before = myList[pos - 1]
    after = myList[pos]
    if after - myNumber < myNumber - before:
        return pos
    else:
        return pos - 1
def calCord(latarray, longarray):
    xaxis = []
    yaxis = []
    for i in range(latarray.shape[0]): 
        lat = latarray[i]
        long = longarray[i]
        
        latMid = (lat+minlat )/2.0


        m_per_deg_lat = (111132.954 - 559.822 * math.cos( 2.0 * latMid ) + 1.175 * math.cos( 4.0 * latMid)) / 1e5
        m_per_deg_lon = ((3.14159265359/180 ) * 6367449 * math.cos ( latMid ))/1e5

        deltaLat = math.fabs(lat - minlat)
        deltaLon = math.fabs(long - minlong)
        
        xaxis.append(deltaLat * m_per_deg_lat)
        yaxis.append(deltaLon * m_per_deg_lon )
    
    return np.array(xaxis), np.array(yaxis) 

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

def DepthNorm(name, par, depth):
    keys = np.load(data_path + name + '_keys.npy')
    means = np.load(data_path + name+ '_means.npy')
    stds = np.load(data_path + name + '_stds.npy')
    
    feature_norm = []
    for i in range(len(par)):
        depth_val = -depth[i]
        if not np.isnan(depth_val):
            indices = take_closest(keys, depth_val)
            #print(indices)
        else:
            indices = take_closest(keys, -1)
        #print(len(indices))
        #print(indices)
        #assert len(indices) >= 1
                                                    
        mean_val = means[indices]
        std_val = stds[indices]
        new_val = (par[i] - mean_val) / (std_val + 1e-3)
        feature_norm.append(new_val)
        
    
    return feature_norm

def findLargestIndex(time, specific_index):
    index = specific_index - 1
    while True:
        if index < 0 or time[index] < time[specific_index]:
            break
        else:
            index = index - 1
    return index

def NormalizeDataMultiDim(data):
    for i in range(data.shape[1]):
        if not np.isnan ((np.max(data[:, i]) - np.min(data[:,i]))):
            data[:,i] = (data[:,i] - np.min(data[:,i])) / (np.max(data[:, i]) - np.min(data[:,i]))
    return data

def shuffle_data_label(data, label):
    shuffler = np.random.permutation(len(label))
    data_shuffled = data[shuffler]
    label_shuffled = label[shuffler]
    return data_shuffled, label_shuffled

def prepareDataset(features_st,label, TIME_WINDOW): 
    time = features_st[:, -1]
    features_concat = []
    label_concat = []
    for i in range(features_st.shape[0]):
        idx_end = findLargestIndex(time, i)
        idx_start = idx_end + 1  - TIME_WINDOW
        if idx_start > 0:
            features_neigh = features_st[idx_start:idx_end + 1, :]
            features_i = features_st[i,:]
            features_i = np.expand_dims(features_i, axis = 0)
            #features_i = np.expand_dims(features_i, axis = 0)
            #features_neigh = np.expand_dims(features_neigh, axis = 0)
            features_concat_i = np.concatenate((features_i, features_neigh), axis = 0)
            features_concat.append(features_concat_i)
            label_concat.append(label[i])
    return np.array(features_concat),np.array(label_concat)

home_path = '/home/whe2/STGNN/experiment/'
with open(home_path + 'data/paired_dataset.pkl', 'rb') as f:
    paired_data = pickle.load(f)
file_errors_location = home_path + 'data/PinellasMonroeCoKareniabrevis 2010-2020.06.12.xlsx'
df = pd.read_excel(file_errors_location)
for name in ['par', 'chlor_a', 'Rrs_443','Rrs_469','Rrs_488', 'Kd_490', 'nflh']:
    df[name] = 0
df = df.sort_values('Sample Date')
df_unix_sec = (pd.to_datetime(df['Sample Date'])).astype(int)/10**9
df_unix_sec = df_unix_sec.to_numpy()
#df_unix = (df_unix_sec - df_unix_sec[0])/(df_unix_sec[3] - df_unix_sec[0])
#df['Sample Date'] = df_unix
#da_new = df[['Sample Date', 'Sample Depth (m)', 'Latitude','Longitude','Karenia brevis abundance (cells/L)' ]]
#da_new = da_new.to_numpy()

time_df = df_unix_sec
lat_df = df['Latitude']
long_df = df['Longitude']
concen_df = df['Karenia brevis abundance (cells/L)']

length = len(time_df)
features_df = df['par'].to_numpy().reshape(length,1)
for name in [ 'chlor_a', 'Rrs_443','Rrs_469','Rrs_488', 'Kd_490', 'nflh','Karenia brevis abundance (cells/L)']:
    features_df = np.concatenate((features_df, df[name].to_numpy().reshape(length,1)),axis = 1)



paired_data = paired_data.sort_values('Sample Date')
depth = paired_data['Sample Depth']
time_paired = paired_data['Sample Date']
time_paired = (pd.to_datetime(time_paired)).astype(int)/10**9
time_paired = time_paired.to_numpy() 



lat_paired = paired_data['Latitude']
long_paired = paired_data['Longitude']
lat_paired = lat_paired.to_numpy()
long_paired = long_paired.to_numpy()
maxlat_paired = np.max(lat_paired)
minlat_paired = np.min(lat_paired)
maxlong_paired = np.max(long_paired)
minlong_paired = np.min(long_paired)


par = paired_data['par']
par = par.to_numpy()
chlor_a = paired_data['chlor_a']
chlor_a = chlor_a.to_numpy()
Rrs_443 = paired_data['Rrs_443']
Rrs_443 = Rrs_443.to_numpy()
Rrs_469 = paired_data['Rrs_469']
Rrs_469 = Rrs_469.to_numpy()
Rrs_488 = paired_data['Rrs_488']
Rrs_488 = Rrs_488.to_numpy()
Kd_490 = paired_data['Kd_490']
Kd_490 = Kd_490.to_numpy()
nflh = paired_data['nflh']
nflh = nflh.to_numpy()

data_path = home_path + 'data/depth_stats/'
par_norm = DepthNorm('par',par,depth)
chlor_a_norm = DepthNorm('chlor_a', chlor_a, depth)
Rrs_443_norm = DepthNorm('Rrs_443', Rrs_443, depth)
Rrs_469_norm = DepthNorm('Rrs_469', Rrs_469, depth)
Rrs_488_norm = DepthNorm('Rrs_488', Rrs_488, depth)
Kd_490_norm = DepthNorm('Kd_490', Kd_490, depth)
nflh_norm = DepthNorm('nflh', nflh, depth)

concen_paired = paired_data['Red Tide Concentration']

features_paired = np.stack((par_norm, chlor_a_norm, Rrs_443_norm,Rrs_469_norm,Rrs_488_norm, Kd_490_norm, nflh_norm,concen_paired), axis = 1) 
#features = NormalizeDataMultiDim(features)
#features = np.stack((par, chlor_a, Rrs_443,Rrs_469,Rrs_488, Kd_490, nflh), axis = 1) 

time = np.concatenate((time_df, time_paired), axis = 0)
lat = np.concatenate((lat_df, lat_paired), axis = 0)
long = np.concatenate((long_df, long_paired), axis = 0)
features = np.concatenate((features_df, features_paired), axis = 0)
data_array = np.stack((time, lat, long),axis = 1) 
data_array = np.concatenate((data_array,features), axis = 1)
ind = np.argsort( data_array[:,0] )
data_array = data_array[ind]

time = data_array[:, 0]
time = (time  -  time[0]) / (time[3] - time[0])

lat = data_array[:,1]
long = data_array[:, 2]
features = data_array[:,3:]

concen = np.copy(features[:, -1])
features[:, -1] = np.log(concen + 1)
#concen_mean= np.mean(concen)
#concen_std = np.std(concen)
#concen = (concen - concen_mean) / concen_std

label = np.copy(concen)


if regression:
    label = np.log10(label+1)
    #label = label/np.max(label)
else:
    label = (label > 1e5)
    label = label.astype('uint8')



minlat = np.min(lat)
minlong = np.min(long)
xaxis, yaxis = calCord(np.copy(lat), np.copy(long))
locations = np.stack((xaxis, yaxis), axis = 1)
locations_latlong =  np.stack((lat, long), axis = 1)


def RemoveNan(features, time, locations, locations_latlong, label,concen):
    features_new = []
    time_new = []
    locations_new = []
    locations_latlong_new = []
    label_new = []
    concen_new = []
    
    for i in range(features.shape[0]):
        #flag = 1
        for j in range(features.shape[1]):
            if np.isnan(features[i,j]):
                break;
            if (j == features.shape[1] - 1):
                features_new.append(features[i,:])
                time_new.append(time[i])
                locations_new.append(locations[i,:])
                locations_latlong_new.append(locations_latlong[i,:])
                label_new.append(label[i])
                concen_new.append(concen[i])
    return np.array(features_new), np.array(time_new), np.array(locations_new),np.array(locations_latlong_new), np.array(label_new), np.array(concen)

features, time, locations, locations_latlong, label,concen = RemoveNan(features, time, locations,locations_latlong, label,concen)  

time = time.reshape(len(time), 1)
features_st = np.concatenate((features, locations_latlong, time), axis = 1) # -1 time, -2 and -3 locations, -4 concen

features_data, label_data = prepareDataset(features_st, label, TIME_WINDOW) 
### final data shape: N*301*11, first 7 feature are spectral feature; then cell concentration; location1, location2, time. 


Data split and augmentation

In [ ]:
###

train_instance = int(np.shape(features_data)[0]*0.7)
val_instance = int(np.shape(features_data)[0]*0.8)

train_features_data = features_data[:train_instance]
train_label = label_data[:train_instance]

val_features_data = features_data[train_instance:val_instance]
val_label = label_data[train_instance:val_instance]

test_features_data = features_data[val_instance:]
test_label = label_data[val_instance:]

train_features_data, train_label = data_augumentation(train_features_data, train_label, aug_times = 10)